In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/her-will-ai-for-equality-datathon-2024/sample_submission.csv
/kaggle/input/her-will-ai-for-equality-datathon-2024/train.csv
/kaggle/input/her-will-ai-for-equality-datathon-2024/test.csv


In [4]:
df_train = pd.read_csv("/kaggle/input/her-will-ai-for-equality-datathon-2024/train.csv")
df_test = pd.read_csv("/kaggle/input/her-will-ai-for-equality-datathon-2024/test.csv")
df_sub = pd.read_csv("/kaggle/input/her-will-ai-for-equality-datathon-2024/sample_submission.csv")

In [5]:
df_train.head()

,id,passage,y
0,0,Women are always the ones struggling in math c...,1
1,1,Men in education often just boss around and ne...,0
2,2,Non-binary students are just confused about th...,2
3,3,Science classrooms are designed to be neutral ...,3
4,4,Male teachers often believe they are superior ...,0


In [16]:
import tensorflow as tf
from transformers import BertTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tokenizer

ModuleNotFoundError: No module named 'tokenizer'

In [14]:
# Define the maximum length for padding
max_len = 160

# Define a function for tokenizing the text
def tokenize_data(texts, max_len):
    # Tokenize and prepare inputs for BERT
    inputs = tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"]


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
# Apply tokenization on the 'passage' column
input_ids, attention_masks, token_type_ids = tokenize_data(df_train["passage"], max_len)


TypeError: Unrecognized keyword arguments: {'max_length': 160, 'padding': 'max_length', 'truncation': True, 'return_tensors': 'tf'}

In [4]:
df_train.head(5)

,id,passage,y
0,0,Women are always the ones struggling in math c...,1
1,1,Men in education often just boss around and ne...,0
2,2,Non-binary students are just confused about th...,2
3,3,Science classrooms are designed to be neutral ...,3
4,4,Male teachers often believe they are superior ...,0


In [5]:
df_sub.head(5)

,id,y_pred,parameters
0,7000,0,1.000000e+10
1,7001,0,1.000000e+10
2,7002,0,1.000000e+10
3,7003,0,1.000000e+10
4,7004,0,1.000000e+10


In [39]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

In [7]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
#import tokenization
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [10]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [37]:
def build_model(bert_layer, max_len=160, num_classes=4):
    # Define input layers for tokenized inputs
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    # BERT layer: use bert_layer and pass inputs as a single list
    bert_inputs = [input_word_ids, input_mask, segment_ids]
    bert_outputs = bert_layer(bert_inputs)

    # Using 'sequence_output' as a fallback if 'pooled_output' causes issues
    pooled_output = bert_outputs["pooled_output"] if "pooled_output" in bert_outputs else bert_outputs[0][:, 0]

    # Classification layer (softmax for multi-class)
    output = Dense(num_classes, activation="softmax")(pooled_output)

    # Build and compile the model
    model = Model(inputs=bert_inputs, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [12]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [28]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [29]:
train_input = bert_encode(df_train.passage.values, tokenizer, max_len=160)
test_input = bert_encode(df_test.passage.values, tokenizer, max_len=160)
train_labels = df_train['y'].values

In [36]:
print(hub.__version__)

0.16.1


In [40]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=True)
model = build_model(bert_layer, max_len=160, num_classes=4)
model.summary()

ValueError: Exception encountered when calling layer 'keras_layer_4' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_4' (type KerasLayer):
  • inputs=['<KerasTensor shape=(None, 160), dtype=int32, sparse=None, name=input_word_ids>', '<KerasTensor shape=(None, 160), dtype=int32, sparse=None, name=input_mask>', '<KerasTensor shape=(None, 160), dtype=int32, sparse=None, name=segment_ids>']
  • training=None